In [11]:
import os
from typing import TypedDict, Annotated, List, Dict, Any
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from tavily import TavilyClient
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [12]:
class AgentState(TypedDict):
    messages: Annotated[List[Any], "The conversation messages"]
    query: Annotated[str, "The user's search query"]
    search_results: Annotated[Dict, "Search results from Tavily"]
    final_answer: Annotated[str, "Final answer to the user"]

class SimpleSearchAgent:
    def __init__(self):
        # Initialize Gemini LLM
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            api_key=os.getenv("GEMINI_API_KEY")
        )
      
        # Initialize Tavily client
        self.tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        
        # Create the graph
        self.graph = self._create_graph()
    
    def _tavily_search(self, query: str) -> Dict:
        """Search using Tavily client"""
        try:
            # Use Tavily search with various options
            response = self.tavily_client.search(
                query=query,
                search_depth="advanced",  # or "basic"
                max_results=5,
                include_answer=True,
                include_raw_content=False,
                include_images=False
            )
            
            return response
            
        except Exception as e:
            return {
                "error": f"Search error: {str(e)}",
                "results": [],
                "answer": ""
            }
    
    def _search_node(self, state: AgentState) -> AgentState:
        """Execute Tavily search"""
        query = state["query"]
        search_results = self._tavily_search(query)
        state["search_results"] = search_results
        return state
    
    def _answer_node(self, state: AgentState) -> AgentState:
        """Generate final answer using Gemini with search results"""
        
        query = state["query"]
        search_results = state["search_results"]
        
        # Format search results for the prompt
        results_text = ""
        
        # Check for errors first
        if "error" in search_results:
            results_text = f"Search Error: {search_results['error']}"
        else:
            # Add Tavily's AI answer if available
            if search_results.get("answer"):
                results_text += f"Tavily AI Answer: {search_results['answer']}\n\n"
            
            # Add individual search results
            if search_results.get("results"):
                results_text += "Search Results:\n"
                for i, result in enumerate(search_results["results"], 1):
                    results_text += f"{i}. {result.get('title', 'No title')}\n"
                    results_text += f"   URL: {result.get('url', 'No URL')}\n"
                    results_text += f"   Content: {result.get('content', 'No content')[:300]}...\n"
                    results_text += f"   Score: {result.get('score', 'N/A')}\n\n"
        
        # Create prompt for answer generation
        answer_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a helpful AI assistant. Based on the search results provided, give a comprehensive and accurate answer to the user's question.

                        Guidelines:
                        1. Use the search results as your primary source of information
                        2. Be factual and provide specific details from the search results
                        3. If there's a Tavily AI answer, incorporate it but add additional context from other results
                        4. Structure your answer clearly with relevant details
                        5. Mention sources when citing specific information
                        6. If search results are insufficient, clearly state what information is missing

                        Search Results:
                        {search_results}"""),
                                    ("human", "Question: {query}")
                                ])
        
        answer_chain = answer_prompt | self.llm
        
        try:
            response = answer_chain.invoke({
                "query": query,
                "search_results": results_text
            })
            
            state["final_answer"] = response.content
            
            # Add AI response to messages
            state["messages"].append(AIMessage(content=response.content))
            
        except Exception as e:
            error_msg = f"Error generating answer: {str(e)}"
            state["final_answer"] = error_msg
            state["messages"].append(AIMessage(content=error_msg))
        
        return state
    
    def _create_graph(self) -> StateGraph:
        """Create the simple LangGraph workflow"""
        
        workflow = StateGraph(AgentState)
        
        # Add nodes
        workflow.add_node("search", self._search_node)
        workflow.add_node("answer", self._answer_node)
        
        # Add edges
        workflow.set_entry_point("search")
        workflow.add_edge("search", "answer")
        workflow.add_edge("answer", END)
        
        return workflow.compile()
    
    def search(self, query: str) -> Dict[str, Any]:
        """
        Search and get answer
        
        Args:
            query: The search query
        
        Returns:
            Dictionary with search results and answer
        """
        
        initial_state = {
            "messages": [HumanMessage(content=query)],
            "query": query,
            "search_results": {},
            "final_answer": ""
        }
        
        final_state = self.graph.invoke(initial_state)
        
        return {
            "query": query,
            "search_results": final_state.get("search_results", {}),
            "answer": final_state.get("final_answer", ""),
            "tavily_answer": final_state.get("search_results", {}).get("answer", ""),
            "source_count": len(final_state.get("search_results", {}).get("results", []))
        }
    
    def get_raw_search(self, query: str) -> Dict:
        """
        Get raw Tavily search results without LLM processing
        
        Args:
            query: The search query
            
        Returns:
            Raw Tavily response
        """
        return self._tavily_search(query)

# Additional utility functions for different search types
class TavilySearchUtils:
    def __init__(self, api_key: str):
        self.client = TavilyClient(api_key=api_key)
    
    def basic_search(self, query: str, max_results: int = 3):
        """Quick basic search"""
        return self.client.search(
            query=query,
            search_depth="basic",
            max_results=max_results,
            include_answer=True
        )
    
    def deep_search(self, query: str, max_results: int = 10):
        """Comprehensive research search"""
        return self.client.search(
            query=query,
            search_depth="advanced",
            max_results=max_results,
            include_answer=True,
            include_raw_content=True
        )
    
    def qna_search(self, query: str):
        """Get just the AI-generated answer"""
        response = self.client.qna_search(query=query)
        return response

In [16]:
# Example usage
if __name__ == "__main__":
    # Initialize the agent
    agent = SimpleSearchAgent()
    
    # Example searches
    queries = [
        "I need complete details about the healthcare professional such as contact, email, specilaization, hospital associlated etc.Alessandra Beretta Spedali Civili(BS) Brescia",
    ]
    
    for query in queries:
        print(f"\n{'='*60}")
        print(f"Query: {query}")
        print('='*60)
        
        result = agent.search(query)
        
        print(f"Tavily AI Answer: {result['tavily_answer'][:200]}..." if result['tavily_answer'] else "No direct answer")
        print(f"\nFinal Answer: {result['answer'][:300]}...")
        print(f"Sources found: {result['source_count']}")
        
        # Example of raw search
        print(f"\n--- Raw Tavily Response Preview ---")
        raw_result = agent.get_raw_search(query)
        if raw_result.get("results"):
            first_result = raw_result["results"][0]
            print(f"Top result: {first_result.get('title', 'No title')}")
            print(f"URL: {first_result.get('url', 'No URL')}")


Query: I need complete details about the healthcare professional such as contact, email, specilaization, hospital associlated etc.Alessandra Beretta Spedali Civili(BS) Brescia
Tavily AI Answer: Alessandra Beretta is a healthcare professional at Spedali Civili di Brescia, specializing in nephrology. Contact details are not publicly available. For inquiries, email protocollo.spedalicivilibresc...

Final Answer: Based on the provided search results, Alessandra Beretta is a nephrology specialist at Spedali Civili di Brescia.  However, her contact details are not publicly available.  The general contact email for Spedali Civili di Brescia is protocollo.spedalicivilibrescia@legalmail.it [4].  This email addres...
Sources found: 5

--- Raw Tavily Response Preview ---
Top result: Spedali Civili di Brescia - Healthcare provider - EuroBloodNet
URL: https://eurobloodnet.eu/members/hospital/122/spedali-civili-di-brescia
